# Задача классификации

Очень часто постановка задачи машиного обучения следующая:     
**Написать программу, которая отвечает "Да" или "Нет"** --- то есть, относит каждый объект к положительному ("**Да**" или "**1**") либо к отрицательному ("**Нет**" или "**0**") классу.     
В общем случае, задача, когда надо отнести объект к одному из $n$ классов, называется задачей классификации.      

Далее мы будем работать с уже знакомым вам датасетом из прошлого ноутбука (файл **dataset.csv**).      
#### Задание 1: 
Импортируйте библиотеку pandas, прочитайте датасет в переменную df и выведите первые пять строчек

In [2]:
df.head()

,Gender,Height,Weight
0,Male,187.571423,109.720985
1,Male,174.706036,73.622732
2,Male,188.239668,96.497550
3,Male,182.196685,99.809504
4,Male,177.499762,93.598619


Как можно заметить, в этом файле каждый человек описывается тремя взаимосвязанными параметрами. В прошлый раз мы строили зависимость веса от роста, и использовали пол, как способ улучшить нашу модель.        

А теперь давайте попробуем угадать пол человека по его росту и весу. 

Существует масса алгоритмом классификации. Вот краткий перечень основных групп:
1. Линейные. Мы о них обязательно еще поговорим подробно. Пока только отмечу, что их суть: проведение такой линии (в случае двумерного пространства это линия, в случае многомерного -- гиперплоскость), что объекты одного класса окажутся по одну сторону от этой линии, объекты другого -- по другую. 
2. Метрические. Самый простой способ объяснить людям, далеким от мира computer science (по-русски: информатики), что же такое "машинное обучение". Если мы хотим построить предсказание для нового объекта, мы ищем самый похожий на него из тех, что мы уже видели. И возвращаем ту же метку класса. Если Вася мужского пола и весит 100 кг при росте 2метра, то, наверное, этот неизвестный нам человек с весом 103 кг и ростом 195 см -- тоже мужского пола. Несмотря на свою простоту, эти алгоритмы иногда оказываются на удивление эффективными. 
3. Логические. Пытаются выделить из данных некую закономерность.

В будущем, нам предстоит очень много работать с логическими алгоритмами, точнее, с их подклассом -- алгоритмами, основанными на решающих деревьях. На сегодняшний день, эти алгоритмы являются самыми сильными для многих случаев.      
Ну, и раз уж они основаны на решающих деревьях, то мы начнем с обучения одного решающего дерева.       

Импортируем нужный нам модуль:

In [4]:
from sklearn.tree import DecisionTreeClassifier

#### Задание 2: 
Перед обучением, датасет нужно приготовить: превратить столбец **Gender** в столбец из нулей и единиц. Или сделать новый столбец.      
В прошлом ноутбуке вы это уже делали, повторите сейчас. И выведите первые пять строчек нового датасета.

,Gender,Height,Weight
0,1,187.571423,109.720985
1,1,174.706036,73.622732
2,1,188.239668,96.497550
3,1,182.196685,99.809504
4,1,177.499762,93.598619


Отлично, датасет подготовлен. Сформируем матрицу признаков и положим ее в переменную X:

In [23]:
X = df[['Height', 'Weight']]

X.head()

,Height,Weight
0,187.571423,109.720985
1,174.706036,73.622732
2,188.239668,96.497550
3,182.196685,99.809504
4,177.499762,93.598619


И вектор целевой переменной

In [24]:
y = df[df.columns[df.dtypes != object].drop(['Height', 'Weight'])[0]]

y.head()

0    1
1    1
2    1
3    1
4    1
Name: Gender, dtype: int64

Небольшой оффтоп: что за магия скрывается за командой из предыдущей ячейки?      
Более интересный вопрос: как вообще читать подобные многоэтажные команды?       
Попробую на него ответить: начинаем ее разворачивать. Смотрим на каждый уровень, начиная с самого верхнего и постепенно опускаясь в подробности:
* **y = df[**df.columns[df.dtypes != object].drop(['Height', 'Weight'])[0]**]** -- берем некий столбец (или столбцы) из датафрейма df и кладем его/их в переменную y
* df.columns[df.dtypes != object].drop(['Height', 'Weight'])**[0]** -- берем нулевой (т.е. первый по счету) элемент чего-то (какого-то списка или другого контейнера)
* df.columns[df.dtypes != object]**.drop(['Height', 'Weight'])** -- гуглим, что же делает функция drop, понимаем, что на этом этапе мы выбросили из какого-то pandas-контейнера элементы 'Height', 'Weight'.
* df.columns**[df.dtypes != object]** -- гуглим, что такое df.columns, понимаем, что это просто список (на самом деле, не список, а некий pandas-контейнер) столбцов. Поскольку он является частью библиотеки pandas, он поддерживает много идентичных команд. В том числе, и выбор элементов по логическому условию. Что мы и сделали. 
* **df.dtypes != object** -- гуглим, что такое df.dtypes и что такое object. Узнаем следующее: **df.dtypes** -- команда, возвращающая типы переменных в столбцах датафрейма. Работает не особо информативно, все нечисловые типы обозначает как **object**. Возвращает pandas-столбец (он называется Series) с названиями столбцов в качестве индексов. 

Итак, мы берем список столбцов, выкидываем все, которые содержат нечисловые значения, после чего выкидываем столбцы 'Height', 'Weight'. От получившегося списка столбцов берем самый первый. И кладем этот столбец в переменную y.        
Для чего такая команда: я не знал, как вы назовете новый (или преобразованный) столбец, в котором вместо 'Male' и 'Female' стоят единицы и нули, поэтому написал команду, которая сработает в любом случае (хотя если вы перед нужным столбцом создадите какой-нибудь ненужный столбец из непонятно каких чисел, команда положит в переменную y именно этот ненужный столбец)

Возвращаемся к Data Science.     

Обучим, наконец, наше дерево:

In [25]:
tree = DecisionTreeClassifier()
tree.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

#### Задание 3: 
Получите вектор предсказаний модели на матрице объектов X и положите его в переменную y_pred

Оценим точность модели. Существуют разные способы это сделать, мы сейчас воспользуемся самым простым: метрикой accuracy. Она считается по следующей формуле:
$$
accuracy = \frac{1}{n}\sum_{i=1}^{n}\mathbf{I}(y\_pred_i == y_i )
$$
где:
$$
\mathbf{I}(A) = 
\left\{\begin{matrix} 
1, \text{если А истинно}, \\
0, \text{если А ложно},
\end{matrix}\right.
$$

То есть, это просто доля правильно угаданных ответов

#### Задание 4: 
Посчитайте эту метрику.      
Положите ее в переменную accuracy

In [32]:
accuracy

1.0

Получили 100% точность. То есть, решающее дерево дало правильный ответ во всех случаях.     
Все, мы нашли идеальный алгоритм.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
Курс закончен, расходимся =)           
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        

.        
.        
.        
.        
.        
.        
.        
.        
.        
Ладно, тут есть подвох.        
Подвох заключается в том, что мы проверили точность алгоритма на тех же данных, на которых мы его и обучили. Точность 100% означает лишь, что он заучил все эти данные.       
Как же тогда проверить качество алгоритма?
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
.        
Об этом -- в следующем ноутбуке! Не переключайтесь!           
Шутка, переключитесь ненадолго и решите простую задачку по программированию:           
#### Задание 5: 
Задача "Secret message" с сайта checkio.org:          
<div style="text-align: right"> 
"Где умный человек прячет лист? В лесу. Но что если леса нет? ... Он выращивает лес и прячет его там." 
Гилберт Кит Честертон
</div>


Когда-нибудь пробовали отправить секретное сообщение кому-то не пользуясь услугами почты? Вы можете использовать газету, чтобы рассказать кому-то свой секрет. Даже если кто-то найдет ваше сообщение, легко отмахнуться и сказать что это паранойя и теория заговора. Один из самых простых способов спрятать ваше секретное сообщение это использовать заглавные буквы. Давайте найдем несколько таких секретных сообщений.

Дан кусок текста. Соберите все заглавные буквы в одно слово в том порядке как они встречаются в куске текста.

Например: текст = "**H**ow are you? **E**h, ok. **L**ow or **L**ower? **O**hhh.", если мы соберем все заглавные буквы, то получим сообщение "**HELLO**".


**Входные данные:** Текст как строка (юникод).         
**Выходные данные:** Секретное сообщение как строка или пустая строка.     



In [33]:
def find_message(text):
    return 'my answer'

In [34]:
assert find_message("How are you? Eh, ok. Low or Lower? Ohhh.") == "HELLO", "hello"
assert find_message("hello world!") == "", "Nothing"
assert find_message("HELLO WORLD!!!") == "HELLOWORLD", "Capitals"

AssertionError: hello